In [1]:
import sys
sys.path.append('../../')

## Template - Bias Mitigation Benchmark ([Holistic AI](https://research.holisticai.com))

**Task:** Binary Classification

**Type:** Postprocessing


This notebook is a template for the Bias Mitigation Benchmark. It can be used to mitigate bias in datasets and models. The notebook is based on the [Holistic AI open source library](https://github.com/holistic-ai/holisticai) and follows the bias mitigation benchmark outlined in [Holistic AI](https://research.holisticai.com).

### Template Structure

The template have the following steps:

1. Setup definition: 
    - select a task: `binary_classification`, `multiclass_classification`, `regression`, `clustering`, `recommender`
    - select a type: `inprocessing`, `preprocessing`, `postprocessing`
2. Mitigator class
    - create a class for you custom mitigator
3. Evaluation
    - evaluate your mitigator and compare it with other mitigators
4. Submission
    - do you have good results? Then submit your mitigator to the Bias Mitigation Benchmark


### Step 1: Setup Definition

In [2]:
from holisticai.benchmark.tasks import task_name, get_task

print(task_name)

['binary_classification', 'multiclass_classification', 'regression', 'clustering', 'recommender']


In [3]:
# load a task
task = get_task("binary_classification")

In [4]:
# benchmark for the task by type
data = task.benchmark(type='postprocessing')
data

Dataset,Average AFS,adult,bank_marketing,census_kdd,compas_recidivism,credit_card,diabetes,german_credit,law_school
Mitigator,,,,,,,,,
MLDebiaser,0.891768,0.884651,0.947397,0.960676,0.814375,0.902120,0.804558,0.888181,0.932189
EqualizedOdds,0.879779,0.854772,0.943726,0.960879,0.809373,0.900984,0.800230,0.866197,0.902067
RejectOptionClassification,0.865316,0.880701,0.907784,0.955288,0.821524,0.879057,0.784279,0.765489,0.928405
LPDebiaserBinary,0.847006,0.825449,0.914638,0.905938,0.801134,0.884189,0.790329,0.846687,0.807689
CalibratedEqualizedOdds,0.826822,0.848425,0.942929,0.959960,0.452759,0.887227,0.795279,0.871521,0.856475


### Step 2: Mitigator Class

In [5]:
import numpy as np

from holisticai.utils.transformers.bias import BMPostprocessing as BMPost
from holisticai.utils.transformers.bias import SensitiveGroups

from holisticai.bias.mitigation.postprocessing.ml_debiaser.randomized_threshold.algorithm import RandomizedThresholdAlgorithm
from holisticai.bias.mitigation.postprocessing.ml_debiaser.reduce2binary.algorithm import Reduce2BinaryAlgorithm


class MyPostprocessingMitigator(BMPost):
    """
    This is a class example of Postprocessing Mitigator based on MLDebiaser implemented in holisticai library
    """

    def __init__(
        self,
        gamma=1.0,
        eps=0,
        eta=0.5,
        sgd_steps=10_000,
        full_gradient_epochs=1_000,
        batch_size=256,
        max_iter=5,
        verbose=True,
    ):
        self.gamma = gamma
        self.eps = eps
        self.eta = eta
        self.sgd_steps = sgd_steps
        self.full_gradient_epochs = full_gradient_epochs
        self.batch_size = batch_size
        self.max_iter = max_iter
        self.verbose = verbose
        self.sens_groups = SensitiveGroups()

    def fit(self, y_proba: np.ndarray, group_a: np.ndarray, group_b: np.ndarray):
        """
        Compute parameters for calibrated equalized odds.

        Description
        ----------
        Compute parameters for calibrated equalized odds algorithm.

        Parameters
        ----------
        y_pred : array-like
            Predicted vector (num_examples,).
        group_a : array-like
            Group membership vector (binary)
        group_b : array-like
            Group membership vector (binary)
        Returns
        -------
        Self
        """
        params = self._load_data(y_proba=y_proba, group_a=group_a, group_b=group_b)
        y_proba = params["y_proba"]
        group_a = params["group_a"] == 1
        group_b = params["group_b"] == 1
        num_classes = y_proba.shape[1]
        sensitive_features = np.stack([group_a, group_b], axis=1)
        self.sens_groups.fit(sensitive_features)

        if num_classes > 2:
            self.algorithm = Reduce2BinaryAlgorithm(
                gamma=self.gamma,
                eps=self.eps,
                eta=self.eta,
                num_classes=num_classes,
                sgd_steps=self.sgd_steps,
                full_gradient_epochs=self.full_gradient_epochs,
                batch_size=self.batch_size,
                max_iter=self.max_iter,
                verbose=self.verbose,
            )
        else:
            self.algorithm = RandomizedThresholdAlgorithm(
                gamma=self.gamma,
                eps=self.eps,
                sgd_steps=self.sgd_steps,
                full_gradient_epochs=self.full_gradient_epochs,
                batch_size=self.batch_size,
                verbose=self.verbose,
            )
        return self

    def transform(
        self,
        y_proba: np.ndarray,
        group_a: np.ndarray,
        group_b: np.ndarray,
    ):
        """
        Apply transform function to predictions and likelihoods

        Description
        ----------
        Use a fitted probability to change the output label and invert the likelihood

        Parameters
        ----------
        y_proba : array-like
            Predicted probability matrix (nb_examlpes, nb_classes)
        group_a : array-like
            Group membership vector (binary)
        group_b : array-like
            Group membership vector (binary)
        threshold : float
            float value to discriminate between 0 and 1

        Returns
        -------
        dictionnary with new predictions
        """
        params = self._load_data(y_proba=y_proba, group_a=group_a, group_b=group_b)

        group_a = params["group_a"] == 1
        group_b = params["group_b"] == 1
        y_proba = params["y_proba"]
        sensitive_features = np.stack([group_a, group_b], axis=1)
        p_attr = self.sens_groups.transform(sensitive_features, convert_numeric=True)

        if type(self.algorithm) is Reduce2BinaryAlgorithm:
            # Multiclass classification
            new_y_prob = self.algorithm.predict(y_proba, p_attr)
            new_y_pred = new_y_prob.argmax(axis=-1)
            return {"y_pred": new_y_pred, "y_proba": new_y_prob}
        else:
            # Binary classification
            pred = y_proba[:, 1]  # .argmax(axis=-1)
            pred = (
                2 * pred - 1
            )  # follow author implementation (use prediction and not logit)
            self.algorithm.fit(pred, p_attr)

            new_y_score = self.algorithm.predict(pred, p_attr)
            new_y_pred = np.where(new_y_score > 0.5, 1, 0)
            return {"y_pred": new_y_pred, "y_score": new_y_score}

    def fit_transform(
        self, y_proba: np.ndarray, group_a: np.ndarray, group_b: np.ndarray
    ):
        """
        Fit and transform

        Description
        ----------
        Fit and transform

        Parameters
        ----------
        y_proba : array-like
            Predicted vector (num_examples,).
        group_a : array-like
            Group membership vector (binary)
        group_b : array-like
            Group membership vector (binary)
        Returns
        -------
        dictionnary with new predictions
        """
        return self.fit(
            y_proba,
            group_a,
            group_b,
        ).transform(y_proba, group_a, group_b)


### Step 3: Evaluation

In [6]:
my_mitigator = MyPostprocessingMitigator()

task.run_benchmark(mitigator = my_mitigator, type = 'postprocessing')

Binary Classification Benchmark initialized for MyMitigator


100%|██████████| 2/2 [00:19<00:00,  9.60s/it]


In [11]:
data = task.evaluate_table()

### Step 4: Submission

In [9]:
task.submit()

MyMitigator benchmark submitted
MyMitigator benchmark submitted
https://holistic-ai.com/benchmark/binary_classification
